# 4.2.1 Normal Order and Aplicative Order

## 4.25

```scheme
(define (unless condition
                usual-value
                exceptional-value)
  (if condition
      exceptional-value
      usual-value))

(define (factorial n)
  (unless (= n 1)
          (* n (factorial (- n 1)))
          1))
```

Evalutaing `(factorial 5)` will *not* work in an applicative order language. Both branches of the `unless` expression are evalutated regardless of the predicate value. When `n == 1`, `(* 1 (factorial 0))` will be evaluated instead of terminating using the `usual-value` branch of `1`. This will lead to `(* 0 (factorial -1))` being evaluated and so on, causing the procedure to run infinitely.

In a normal order language, the procedure will work as expected. Since the branches of the `unless` expression are only evaluated when needed, the procedure will terminate when `n == 1` with the `usual-branch` evaluation of `1`. The `exceptional-value` won't be evaluated, preventing the infinite loop encountered in the applicative order language.

## 4.26

 `unless` can be implemented as a special form in the evaluator easily by transforming it into and `if` expression:

In [1]:
(define (unless? exp)
  (tagged-list? exp 'unless))

(define (unless-condition exp)
  (cadr exp))
(define (unless-usual-value exp)
  (caddr exp))
(define (unless-exceptional-value exp)
  (cadddr exp))

(define (unless->if exp)
  (make-if (unless-condition exp)
           (unless-exceptional-value exp)
           (unless-usual-value exp)))

;; within evaluator cond expression
;;((unless? exp)
 ;;(unless->if exp))

Alyssa's argument is correct in that `unless` as a special form can't be passed as an argument to a higher order procedure such as `map`. However, I cannot think of a use case for doing so.

# 4.2.2 An Interpreter with Lazy Evaluation

Following cell contains code collated from the exercises in the previous chapter.

In [1]:
;; allow access to Scheme version of apply under a different name
;; ONLY execute once per jupyter session to avoid using custom apply instead of Scheme
(define apply-in-underlying-scheme apply)

In [99]:
;; evaluator *without* pre-analysis
(define (eval exp env)
  (cond ((self-evaluating? exp)
         exp)
        ((variable? exp)
         (lookup-variable-value exp env))
        ((quoted? exp)
         (text-of-quotation exp))
        ;; assignment/definition recursively call eval to comput
        ;; new value associated with a variable
        ((assignment? exp)
         (eval-assignment exp env))
        ((definition? exp)
         (eval-definition exp env))
        ((if? exp)
         (eval-if exp env))
        ;; transform into applicable procedure within environment
        ((lambda? exp)
         (eval-lambda exp env))
        ;; evaluate sequence of expressions in order
        ((begin? exp)
         (eval-begin exp env))
        ;; case analysis -> nest of if expressions which are then evaluated
        ((cond? exp)
         (eval-cond exp env))
        ((let? exp)
         (eval-let exp env))
        ;; lazy evaluation -> evaluate operator to obtain procedure for apply
        ;; to be able to dispatch on type (primitive/compound)
        ;; operands not evaluated -> create a thunk
        ((application? exp)
         (apply (actual-value (operator exp) env)
                (operands exp)
                env))
        (else
         (error "Unkown expression
                type: EVAL" exp))))

;; retrieve the actualy value of an expression
;; force a thunk to obtain value
(define (actual-value exp env)
  (force-it (eval exp env)))


;; passed unevaluated operand expressions
(define (apply procedure arguments env)
  (cond ((primitive-procedure? procedure)
         (apply-primitive-procedure
          procedure
          ;; evaluate all arguments *before* applying primitive
          ;; primitive procedures are strict
          (list-of-arg-values
           arguments
           env)))
        ;; sequentially eval each expression in procedure body
        ;; extend base environment to include new frame binding
        ;; params of procedure to the arguments it is applied to
        ((compound-procedure? procedure)
         (eval-sequence
          (procedure-body procedure)
          (extend-environment
           (procedure-parameters
            procedure)
           ;; delay all arguments before applying compound
           ;; compound procedures are non-strict
           (list-of-delayed-args
            arguments
            env)
           (procedure-environment
            procedure))))
        (else
         (error "Unkown procedure
                type: APPLY"
                procedure))))

;; produce list of arguments to which a procedure is to be applied
;; evaluate each operand -> return list of results
(define (list-of-arg-values exps env)
  (if (no-operands? exps)
      '()
      ;; call actual-value *not* eval as argument may be a thunk
      (cons (actual-value
             (first-operand exps)
             env)
            (list-of-arg-values
             (rest-operands exps)
             env))))

;; produce list of delayed arguments to which a procedure is to be applied
;; delay each argument instead of evaluating
(define (list-of-delayed-args exps env)
  (if (no-operands? exps)
      '()
      (cons (delay-it
             (first-operand exps)
             env)
            (list-of-delayed-args
             (rest-operands exps)
             env))))

(define (eval-if exp env)
  ;; actual-value must be used to obtain value of predicate expression
  (if (true? (actual-value (if-predicate exp)
                           env))
      (eval (if-consequent exp) env)
      (eval (if-alternative exp) env)))

;; eval a sequence of expressions in order
;; return value of final expression
(define (eval-sequence exps env)
  (cond ((last-exp? exps)
         (eval (first-exp exps) env))
        (else
         (eval (first-exp exps) env)
         (eval-sequence (rest-exps exps)
                        env))))

;; assign value to variable
(define (eval-assignment exp env)
  (set-variable-value!
   (assignment-variable exp)
   (eval (assignment-value exp) env)
   env)
  'ok)

;; define a variable
(define (eval-definition exp env)
  (define-variable!
   (definition-variable exp)
   (eval (definition-value exp) env)
   env)
  'ok)

(define (eval-begin exp env)
  (eval-sequence
   (begin-actions exp)
   env))

(define (eval-cond exp env)
  (eval (cond->if exp) env))

(define (eval-lambda exp env)
  (make-procedure
   (lambda-parameters exp)
   (lambda-body exp)
   env))


;; expressions

(define (self-evaluating? exp)
  (cond ((number? exp) #t)
        ((string? exp) #t)
        (else #f)))

(define (variable? exp) (symbol? exp))

;; quotations: (quote <text-of-quotation>)
(define (quoted? exp)
  (tagged-list? exp 'quote))
(define (text-of-quotation exp)
  (cadr exp))

(define (tagged-list? exp tag)
  (if (pair? exp)
      (eq? (car exp) tag)
      #f))

;; assignments: (set! <var> <value>)
(define (assignment? exp)
  (tagged-list? exp 'set!))
(define (assignment-variable exp)
  (cadr exp))
(define (assignment-value exp) (caddr exp))

;; definitions: (define <var> <value>)
;; OR (define (<var> <param-1> ... <param-n>) <body>)
(define (definition? exp)
  (tagged-list? exp 'define))
(define (definition-variable exp)
  (if (symbol? (cadr exp))
      (cadr exp)
      (caadr exp)))
(define (definition-value exp)
  (if (symbol? (cadr exp))
      (caddr exp)
      (make-lambda
       (cdadr exp) ; formal parameters
       (cddr exp)))) ; body

;; (make-definition 'my-val '() 1)
;; -> (define myval 1)
;; (make-definition 'my-func` '(x y) (display (* x y))
;; -> (define (my-func x y) (display (*x y)))
(define (make-definition identifier params body)
  (if (null? params)
      (list 'define identifier body)
      (list 'define (cons identifier params) body)))

;; lambda expressions: (lambda (<param-1> ... <param-n>) <body>)
(define (lambda? exp)
  (tagged-list? exp 'lambda))
(define (lambda-parameters exp) (cadr exp))
(define (lambda-body exp) (cddr exp))
(define (make-lambda parameters body)
  (cons 'lambda (cons parameters body)))

;; conditionals: (if <predicate> <consequent> <alternative>?)
;; alternative is optional -> default to false
(define (if? exp) (tagged-list? exp 'if))
(define (if-predicate exp) (cadr exp))
(define (if-consequent exp) (caddr exp))
(define (if-alternative exp)
  (if (not (null? (cdddr exp)))
      (cadddr exp)
      'false)) ;; default to false if no alternative
(define (make-if predicate
                 consequent
                 alternative)
  (list 'if
        predicate
        consequent
        alternative))

;; begin: (begin <exp-1> ... <exp-n>)
;; package a sequence of expressions into a single expression
(define (begin? exp)
  (tagged-list? exp 'begin))
(define (begin-actions exp) (cdr exp))
(define (last-exp? seq) (null? (cdr seq)))
(define (first-exp seq) (car seq))
(define (rest-exps seq) (cdr seq))
(define (sequence->exp seq)
  (cond ((null? seq) seq)
        ((last-exp? seq) (first-exp seq))
        (else (make-begin seq))))
(define (make-begin seq) (cons 'begin seq))

;; procedure application: any compound expression not defined as an expression type above
;; (<operator> <operand-1> ... <operand-n>)
(define (application? exp) (pair? exp))
(define (operator exp) (car exp))
(define (operands exp) (cdr exp))
(define (no-operands? ops) (null? ops))
(define (first-operand ops) (car ops))
(define (rest-operands ops) (cdr ops))

;; cond: (cond ((<predicate-1> <result-1> ... <predicate-n> <result-1n) (else <result>))
;; implemented as nested if expressions
(define (cond? exp)
  (tagged-list? exp 'cond))
(define (cond-clauses exp) (cdr exp))
(define (cond-else-clause? clause)
  (eq? (cond-predicate clause) 'else))
(define (cond-predicate clause)
  (car clause))
(define (cond-actions clause)
  (cdr clause))
(define (cond->if exp)
  (expand-clauses (cond-clauses exp)))
(define (expand-clauses clauses)
  (if (null? clauses)
      'false ;; no else clause
      (let ((first (car clauses))
            (rest (cdr clauses)))
        (if (cond-else-clause? first)
            (if (null? rest)
                (sequence->exp
                 (cond-actions first))
                (error "ELSE clause is not last: COND->IF"
                       clauses))
            (make-if (cond-predicate first)
                     (sequence->exp
                      (cond-actions first))
                     (expand-clauses
                      rest))))))

;; extend to include boolean values
(define (self-evaluating? exp)
  (or (number? exp)
      (string? exp)
      (boolean? exp)))

(define (and? exp)
  (tagged-list? exp 'and))
(define (or? exp)
  (tagged-list? exp 'or))
(define (boolean-exps exp)
  (cdr exp))

(define (eval-and exp env)
  (define (eval-exps exp)
    (cond ((last-exp? exp)
           (eval (first-exp exps) env))
          ((eval (first-exp exps) env)
           (eval-exps (rest-exps exp)))
          (else #f)))
  (eval-exps (boolean-exps exp)))

(define (eval-or exp env)
  (define (eval-exps exp)
    (cond ((last-exp? exp)
           (eval (first-exp exps) env))
          ((eval (first-exp exps) env) #t)
          (else (eval-exps (rest-exps exp)))))
  (eval-exps (boolean-exps exp)))


;; OR with derived expressions using nested if
(define (eval-and exp env)
  (eval (and->if (boolean-exps exp) env) env))
(define (eval-or exp env)
  (eval (or->if (boolean-exps exp) env) env))

(define (and->if exp env)
  (cond ((last-exp? exp)
         (eval (first-exp exp) env))
        (else (make-if
               (first-exp exp)
               (and->if (rest-exps exp) env)
               #f))))

(define (or->if exp env)
  (cond ((last-exp? exp)
         (eval (first-exp exp) env))
        (else (make-if
               (first-exp exp)
               #t
               (or->if (rest-exps exp) env)))))

(define (cond-arrow-form? clause)
  (eq? (cadr clause) '=>))
(define (cond-recipient clause)
  (caddr clause))

;; ((<predicate-clauses>) (<result-clauses>)) -> (<result-clauses>)
;; ((<test>) => <recipient>) -> (<recipient> (<test>))
(define (cond-consequent clause predicate)
  (if (cond-arrow-form? clause)
      (list (cond-recipient clause) predicate)
      (sequence->exp (cond-actions clause))))

(define (expand-clauses clauses)
  (if (null? clauses)
      'false ;; no else clause
      (let ((first (car clauses))
            (rest (cdr clauses)))
        (if (cond-else-clause? first)
            (if (null? rest)
                (sequence->exp
                 (cond-actions first))
                (error "ELSE clause is not last: COND->IF"
                       clauses))
            (let ((predicate (cond-predicate first)))
              (make-if predicate
                       (cond-consequent first predicate)
                       (expand-clauses rest)))))))
(define (let? exp)
  (tagged-list? exp 'let))
(define (let-definitions exp)
  (cadr exp))
(define (let-parameters exp)
  (map car (let-definitions exp)))
(define (let-exps exp)
  (map cadr (let-definitions exp)))
(define (let-body exp)
  (cddr exp))

(define (let->combination exp env)
  (cons (make-lambda (let-parameters exp)
                     (let-body exp))
        (let-exps exp)))

(define (eval-let exp env)
  (eval (let->combination exp env) env))

(define (let*? exp)
  (tagged-list? exp 'let*))
(define (let*-definitions exp)
  (cadr exp))
(define (let*-body exp)
  (caddr exp))

(define (let*->nested-lets exp)
  (define (make-let definitions)
    (cond ((last-exp? definitions)
           (append (list 'let (list (first-exp definitions)))
                   (let-body-exp))
           (else (list
                  'let
                  (list (first-exp definitions))
                  (make-let (rest-exps definitions)))))))
  (make-let (let*-let*-definitions)))

(define (eval-let* exp env)
  (eval (let*->nested-lets exp) env))

(define (named-let? exp)
  (and (let? exp)
       (variable? (cadr exp))))

(define (let-parameter-values exp) 
  (map cadr (let-definitions exp)))

;; exp is the named-let stripped of the let symbol
;; to allow original let selectors to be used
(define (named-let->combination exp)
  (let ((identifier (car exp)))
    (display identifier)
    (make-begin
     (list
      (make-definition identifier '()
            (make-lambda (let-parameters exp)
                         (let-body exp)))
      (cons identifier (let-parameter-values exp))))))

(define (let->combination exp env)
  (if (named-let? exp)
      (named-let->combination (cdr exp))
      (cons (make-lambda (let-parameters exp)
                         (let-body exp))
            (let-exps exp))))

;; evaluator data structures

;; anything that is not explicit false is true
(define (true? x)
  (not (eq? x #f)))
(define (false? x)
  (eq? x #f))

;; compound procedures: (procedure (<param-1>...<param-n>) <body> <env>)
(define (make-procedure parameters body env)
  (list 'procedure parameters body env))
(define (compound-procedure? p)
  (tagged-list? p 'procedure))
(define (procedure-parameters p)
  (cadr p))
(define (procedure-body p)
  (caddr p))
(define (procedure-environment p)
  (cadddr p))

;; operations on environments

;; environment: list of frames
(define (enclosing-environment env) (cdr env))
(define (first-frame env) (car env))
(define the-empty-environment '())

;; frame: pair of lists (<variables> <values>)
(define (make-frame variables values)
  (cons variables values))
(define (frame-variables frame)
  (car frame))
(define (frame-values frame)
  (cdr frame))
(define (add-binding-to-frame! var val frame)
  (set-car! frame (cons var (car frame)))
  (set-cdr! frame (cons val (cdr frame))))

;; extend environment: add a new frame of variables + values to an environment
(define (extend-environment vars vals base-env)
  (if (= (length vars) (length vals))
      (cons (make-frame vars vals) base-env)
      (if (< (length vars) (length vals))
          (error "Too many arguments supplied"
                 vars
                 vals)
          (error "Too few arguments supplied"
                 vars
                 vals))))

;; frame: list of (<variable> <value>) pairs
(define (make-frame variables values)
  (map cons variables values))
(define (frame-variables frame)
  (map car frame))
(define (frame-values frame)
  (map cdr frame))
(define (add-binding-to-frame! var val frame)
  (set-car! frame (cons var val)))

(define (scan-env var env found-in-frame not-found-in-frame)
    (if (eq? env the-empty-environment)
      (error "SCAN-ENV" "Unbound variable ~a" var)
  (let ((frame (first-frame env)))
    (define (scan vars vals)
      (cond ((null? vars)
             (not-found-in-frame frame))
            ((eq? var (car vars))
             (found-in-frame vars vals frame))
            (else (scan (cdr vars)
                        (cdr vals)))))

      (scan (frame-variables frame)
            (frame-values frame)))))

(define (lookup-variable-value var env)
  (scan-env var env
            (lambda (vars vals frame) (car vals))
            (lambda (frame) (lookup-variable-value var (enclosing-environment env)))))

(define (set-variable-value! var val env)
  (scan-env var env
            (lambda (vars vals frame) (add-binding-to-frame! var val frame))
            (lambda (frame) (set-variable-value! var val (enclosing-environment env)))))

(define (define-variable! var val env)
  (scan-env var env
            (lambda (vars vals frame) (add-binding-to-frame! var val frame))
            (lambda (frame) (add-binding-to-frame! var val frame))))

(define (make-unbound! var env)
  (scan-env var env
            (lambda (vars vals frame)
              (set-car! vars '())
              (set-car! vals '())
              #t)
            #f))

;; running the evaluator as a program

(define primitive-procedures
  (list (list 'car car)
        (list 'cdr cdr)
        (list 'cons cons)
        (list 'null? null?)
        (list '+ +)
        (list '- -)
        (list '* *)
        (list '/ /)
        (list '= =))) ;; other primitives can be added here

(define (primitive-procedure-names)
  (map car primitive-procedures))

(define (primitive-procedure-objects)
  (map (lambda (proc)
         (list 'primitive (cadr proc)))
       primitive-procedures))

(define (primitive-procedure? proc)
  (tagged-list? proc 'primitive))

(define (primitive-implementation proc)
  (cadr proc))

(define (apply-primitive-procedure proc args)
  (apply-in-underlying-scheme
   (primitive-implementation proc) args))

(define (setup-environment)
  (let ((initial-env
         (extend-environment
          (primitive-procedure-names)
          (primitive-procedure-objects)
          the-empty-environment)))
    ;; bind true/false for use in evaluating expressions
    (define-variable! 'true #t initial-env)
    (define-variable! 'false #f initial-env)
    initial-env))

(define the-global-environment
  (setup-environment))


;; prompts indicate lazy evaluator
(define input-prompt ";;; L-Eval input:")
(define output-prompt ";;; L-Eval value:")

(define (driver-loop)
  (prompt-for-input input-prompt)
  (let ((input (read)))
    (let ((output
           (actual-value ;; use actual-value instead of eval
            input
            the-global-environment)))
      (announce-output output-prompt)
      (user-print output)))
  (driver-loop))

(define (prompt-for-input string)
  (newline) (newline)
  (display string) (newline))

(define (announce-output string)
  (newline) (display string) (newline))

;; avoid printing environment part of a compound procedure
;; environment may be very large or contain cycles
(define (user-print object)
  (if (compound-procedure? object)
      (display
       (list 'compound-procedure
             (procedure-parameters object)
             (procedure-body object)
             '<procedure-env>))
      (display object)))

;; data as programs

;; calysto-scheme doesn't have filter built in
;; name filter-seq to prevent collision in other scheme implementations
(define (filter-seq pred? seq)
  (if (null? seq)
      '()
      (let ((first (car seq))
            (rest (cdr seq)))
        (if (pred? first)
            (cons first (filter-seq pred? rest))
            (filter-seq pred? rest)))))

;; using abstractions from ex 4.12
(define (lookup-variable-value var env)
  (scan-env var env
            (lambda (vars vals frame)
              (let ((val (car vals)))
                (if (eq? val '*unassigned*)
                    (error "unassigned variable" (car vars))
                    val)))
            (lambda (frame) (lookup-variable-value var (enclosing-environment env)))))

;; find all defines in proc-body
;; create let statement assigning all define vars to '*unassigned*
;; create set! statements assgning define expressions to vars
(define (scan-out-defines proc-body)
  (let ((defs (body-definitions proc-body)))
    (if (null? defs)
        proc-body
        (list (append (list 'let (unassigned-defines defs))
                      (unassigned-defines-assignments defs)
                      (scanned-out-body proc-body))))))

(define (body-definitions body)
  (filter-seq definition? body))

(define (scanned-out-body body)
  (filter-seq (lambda (exp) (not (definition? exp))) body))

(define (unassigned-defines defines)
  (map (lambda (var) (list var '*unassigned*))
       (map definition-variable defines))) 

(define (unassigned-defines-assignments defines)
  (map (lambda (def)
         (list 'set!
               (definition-variable def)
               (definition-value def)))
       defines))

;; installed into make-procedure
;; only called once when the lambda is first evaluated
;; installing in procedure-body would call scan-out-defines
;; each time the procedure is called
(define (make-procedure parameters body env)
  (list 'procedure parameters (scan-out-defines body) env))

;; transform letrec into a combination in the same manner as scan-out-defines
(define (letrec? exp)
  (tagged-list? exp 'letrec))
(define (letrec-declarations exp)
  (cadr exp))
(define (letrec-variable declaration)
  (car declaration))
(define (letrec-value declaration)
  (cadr declaration))
(define (letrec-body exp)
  (caddr exp))

(define (letrec->combination exp)
  (let ((decs (letrec-declarations exp)))
    (if (null? decs)
      exp
      (append (list 'let (unassigned-declarations decs))
                    (unassigned-declaration-assignments decs)
                    (letrec-body exp)))))

(define (unassigned-declarations declarations)
  (map (lambda (var) (list var '*unassigned*))
       (map letrec-variable declarations)))

(define (unassigned-declaration-assignments declarations)
  (map (lambda (dec)
         (list 'set!
               (letrec-variable dec)
               (letrec-value dec)))
       declarations))

;; helper to evaluate forms
(define (interpret exp)
  (let ((result (eval exp the-global-environment)))
    (if (thunk? result)
        (force-it result)
        result)))

### Representing Thunks

In [3]:
(define (delay-it exp env)
  (list 'thunk exp env))
(define (thunk? obj)
  (tagged-list? obj 'thunk))
(define (thunk-exp thunk)
  (cadr thunk))
(define (thunk-env thunk)
  (caddr thunk))

;; thunk memoized after first evaluation
(define (evaluated-thunk? obj)
  (tagged-list? obj 'evaluated-thunk))

(define (thunk-value evaluated-thunk)
  (cadr evaluated-thunk))

(define (force-it obj)
  (cond ((thunk? obj) 
         (let ((result ;; evaluate thunk
                (actual-value
                 (thunk-exp obj)
                 (thunk-env obj))))
           ;; memoize result
           (set-car! obj 'evaluated-thunk)
           ;; replace exp with result of evaluation
           (set-car! (cdr obj) result)
           ;; remove env -> no longer needed
           (set-cdr! (cdr obj) '())
           result))
        ;; return memoized result
        ((evaluated-thunk? obj)
         (thunk-value obj))
        ;; not a thunk
        (else obj)))

## 4.27

```scheme
(define count 0)
(define (id x) (set! count (+ count 1)) x)

(define w (id (id 10)))

;;; L-Eval input:
count

;;; L-Eval value:
1 ;; (1)

;;; L-Eval input:
w

;;; L-Eval value:
10 ;; (2)

;;; L-Eval input:
count

;;; L-Eval value:
2 ;; (3)
```

- (1): When `w` is defined, `id` is forced *once*, incrementing `count` from `0` to `1`. `id` is forced in the outer call, but it's argument of `(id 10)` is delayed.`
- (2): `id` returns the value of the argument `x` passed in to it. Evaluating `w` forces both calls made to `id` in `w`. The result of the inner call, `10`, is returned from the outer cal.
- (3): While evaluating `w` in (2), `count` is incremented again due to the application of `id`.

## 4.28

Arguments to a compound procedure are delayed. If a delayed argument is not forced during application and is then used *as if it were* within the body of the procedure an error will be thrown:

```scheme
(define (increment)
  (lambda (x) (+ x 1))
  
(define (do-the-func func x)
  (func x))
 
(do-the-func increment 10)
;; ERROR
```

The arguments to `do-the-func` are delayed, becoming thunks. During application of `do-the-func`, if `eval` is used instead of `actual-value` the thunks are passed in to the procedure body and not the values they represent. Therefore, the application of `(func x)` within `do-the-func` will cause an error as `func` is a thunk and not the `lambda` function that `increment` will return.

## 4.29

Recursive programs will run much more slowly without memoization as each procedure call will require the actual-value of the procedure to be obtained

In [34]:
;; returns the time taken to execute an expression in seconds
(define-syntax time 
  [(time ?exp) (let ((start (current-time)))
                 ?exp
                 (- (current-time) start))])

;; 'save' force-it with memoization
(define force-it-memo force-it)

;; remove memoization
(define (force-it-non-memo obj)
  (if (thunk? obj)
      (actual-value
       (thunk-exp obj)
       (thunk-env obj))
      obj))


(interpret
 '(define (factorial n)
     (if (= n 0) 1
         (* n (factorial (- n 1))))))

(define test-proc '(factorial 100))

;; remove memoization
(define force-it force-it-non-memo)
(display "Without memoization: ")
(display (time (interpret test-proc)))(newline)

;; reinstate memoization
(define force-it force-it-memo)
(display "With memoization: ")
(display (time (interpret test-proc)))(newline)

Without memoization: 8.401825904846191
With memoization: 0.3365287780761719


With memoization:
```scheme
(define (square x) (* x x))

;;; L-Eval input:
(square (id 10))

;;; L-Eval value:
100

;;; L-Eval input:
count

;;; L-Eval value:
1
```

Without memoization:
```scheme
(define (square x) (* x x))

;;; L-Eval input:
(square (id 10))

;;; L-Eval value:
100

;;; L-Eval input:
count

;;; L-Eval value:
2
```

Within the `square` procedure, `(* x x)` uses the value of `x` twice. When memoized, `x` is computed only once, whereas without memoization `x` is computed twice (once for each use of `x`). Since `x` is bound to `(id 10)`, evaluating it causes `count` to be incremented. As such, `count` is incremented once with memoization and twice without memoization.

## 4.30

1. The definition of `for-each` given uses `begin` to call a procedure for each item in a list. `begin` will be evaluated as a sequence, wherein `eval` is called on each expression within the sequence. The `lambda` Ben passes as the `proc` argument to `for-each` uses the *primitive* `display` procedure. Primitive procedures cause their arguments (in this case a thunk) to be forced during evaluation. This means that `for-each` will produce the correct output, as `display` will be called with forced arguments for each item in the sequence.

2. 

#### Original `eval-sequence`:
```scheme
(p1 1)
;; (1 2)

(p2 1)
;; 1
```

In [48]:
(interpret '(define (p1 x)
              (set! x (cons x '(2))) x))

(interpret '(p1 1))

(1 2)

In [100]:
(interpret '(define (p2 x)
              (define (p e) e x)
              (p (set! x (cons x '(2))))))

(interpret '(p2 1))

1

In [54]:
(let? '(let ((p *unassigned*)) (set! p (lambda (e) e x)) (p (set! x (cons x (quote (2)))))))

#t